In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!pip install openai
!pip install tiktoken

In [ ]:
import openai

OPENAI_API_KEY = "sk-XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
openai.api_key = OPENAI_API_KEY

if OPENAI_API_KEY is not None:
    openai.api_key = OPENAI_API_KEY
    print ("OPENAI_API_KEY is ready")
else:
    print ("OPENAI_API_KEY environment variable not found")

In [ ]:
import tiktoken as tiktoken
import time


def limit_tokens_from_string(string: str, model: str, limit: int) -> str:
    """Limits the string to a number of tokens (estimated)."""

    try:
        encoding = tiktoken.encoding_for_model(model)
    except:
        encoding = tiktoken.encoding_for_model('gpt2')  # Fallback for others.

    encoded = encoding.encode(string)

    return encoding.decode(encoded[:limit])

In [ ]:
LLM_MODEL = "gpt-3.5-turbo"
OPENAI_TEMPERATURE = float(0.2)

In [ ]:
def openai_call(
    system_prompt: str,
    prompt: str,
    model: str = LLM_MODEL,
    temperature: float = OPENAI_TEMPERATURE,
    max_tokens: int = 100,
):
    while True:
        try:
            if not model.lower().startswith("gpt-"):
                # Use completion API
                response = openai.Completion.create(
                    engine=model,
                    prompt=prompt,
                    temperature=temperature,
                    max_tokens=max_tokens,
                    top_p=1,
                    frequency_penalty=0,
                    presence_penalty=0,
                )
                return response.choices[0].text.strip()
            else:
                # Use 4000 instead of the real limit (4097) to give a bit of wiggle room for the encoding of roles.
                # TODO: different limits for different models.

                trimmed_prompt = limit_tokens_from_string(prompt, model, 4000 - max_tokens)

                # Use chat completion API
                messages = [{"role": "system" , "content": system_prompt},{"role": "system", "content": trimmed_prompt}]
                response = openai.ChatCompletion.create(
                    model='gpt-3.5-turbo',
                    messages=messages,
                    temperature=temperature,
                    max_tokens=2000,
                    n=1,
                    stop=None,
                )
                return response.choices[0].message.content.strip()
        except openai.error.RateLimitError:
            print(
                "   *** The OpenAI API rate limit has been exceeded. Waiting 10 seconds and trying again. ***"
            )
            time.sleep(10)  # Wait 10 seconds and try again
        except openai.error.Timeout:
            print(
                "   *** OpenAI API timeout occurred. Waiting 10 seconds and trying again. ***"
            )
            time.sleep(10)  # Wait 10 seconds and try again
        except openai.error.APIError:
            print(
                "   *** OpenAI API error occurred. Waiting 10 seconds and trying again. ***"
            )
            time.sleep(10)  # Wait 10 seconds and try again
        except openai.error.APIConnectionError:
            print(
                "   *** OpenAI API connection error occurred. Check your network settings, proxy configuration, SSL certificates, or firewall rules. Waiting 10 seconds and trying again. ***"
            )
            time.sleep(10)  # Wait 10 seconds and try again
        except openai.error.InvalidRequestError:
            print(
                "   *** OpenAI API invalid request. Check the documentation for the specific API method you are calling and make sure you are sending valid and complete parameters. Waiting 10 seconds and trying again. ***"
            )
            time.sleep(10)  # Wait 10 seconds and try again
        except openai.error.ServiceUnavailableError:
            print(
                "   *** OpenAI API service unavailable. Waiting 10 seconds and trying again. ***"
            )
            time.sleep(10)  # Wait 10 seconds and try again
        else:
            break

In [ ]:
!pip install -U torch transformers tokenizers accelerate

In [ ]:
from transformers import AutoTokenizer, LlamaForCausalLM
import json

In [ ]:
MODEL = 'llSourcell/medllama2_7b'

model = LlamaForCausalLM.from_pretrained(MODEL)
tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [ ]:
# JSON 파일 경로 설정
json_file_path = '/content/gdrive/My Drive/Colab Notebooks/lab/data/qafilejson_4.json'

# JSON 파일 읽기
with open(json_file_path, 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)

In [ ]:
# 반복문을 통해 instruction을 가져와서 언어모델을 실행
jsonl_file_path = '/content/gdrive/My Drive/Colab Notebooks/lab/data/output.json'

for item in data:
    prompt = item["prompt"]


    # 영어로 번역해서 질문하기
    system_prompt1 = "You are an agent that generically translate incoming prompts as English prompts."
    prompt1 = f"""
        Please modify the following question into a translated one.
        Here is an example.

        ```
        Question: 전립선암에 걸리면 소변을 볼 때 어떤 문제가 있나요?
        After : What are the problems with urinating if you have prostate cancer?
        ```

        Question: {prompt}
        After:

        """
    response1 = openai_call(system_prompt1, prompt1, max_tokens=200)
    #print(response1)

    # prompt에 번역된 reponse1을instruction에 기입
    prompt2 = f"""
    instruction: {response1}
    input :
    output:
    """

    inputs = tokenizer(prompt2, return_tensors="pt")

    # Generate
    generate_ids = model.generate(inputs.input_ids, max_length=200)
    result = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

    with open(jsonl_file_path, 'a', encoding='utf-8') as jsonl_file:
        jsonl_file.write(json.dumps(result, ensure_ascii=False) + '\n')
